In [ ]:
# # Check if the protected ASes correlated with top finance and insurance companies of the top 8 countries

In [ ]:
# Country specific AS catgories
import pandas as pd
import os
import re
import glob

# countries = pd.read_csv("../data/final_confirmed_customer_ases_location_all_group_2024.csv")
# sorted_countries = countries.sort_values(by = ['value'], ascending=False)
# sorted_countries[0:20]
# Top 20 countries are US, GB, AU, DE, HK, CA, SG, CH, FR, BR, IT, NL, NZ, IN, TW, JP, ID, SE, and AT.
# No. of ASNs on those countries are 643, 96, 79, 58, 45, 42, 32, 31, 30, 27, 27, 24, 21, 20, 17, 15, 14, 14, and 13.

# Define regex pattern
pattern = r"^final_confirmed_customer_ases_location_\d+_2024\.csv$"
directory = "/home/shyam/jupy/ddos_scrubber/data"

country_code = "NL"

csv_files = [f for f in os.listdir(directory) if re.match(pattern, f)]

# Initialize an empty list to store DataFrames
dataframes = []

# Read each CSV file and append it to the list
for file in csv_files:
    df = pd.read_csv(directory+"/"+file)
    
    # Example string

    # Use regex to extract the number between "ases_location_" and "_2024"
    match = re.search(r'final_confirmed_customer_ases_location_(\d+)_2024\.csv', file)
    scrubber_asn = match.group(1)
    
    # Remove rows which do not have country code
    filtered_df = df[~df['country'].str.contains(r'^(?:\?|.*[\t ]\?.*)', na=False)].copy() # Make a copy here
    
    # Use .loc to avoid the warning and replace spaces in 'country' column
    filtered_df.loc[:, 'country'] = filtered_df['country'].apply(lambda x: x.replace(' ', ''))

    filtered_df[['country']] = filtered_df[['country']].apply(lambda x: x.astype(str).str[0:2])
    
    # Remove tabs and spaces from column names
    filtered_df.columns = filtered_df.columns.str.replace(r'[\t ]', '', regex=True)
    
    ases = filtered_df[filtered_df["country"] == country_code]
    ases_copy = ases.copy()

    if not ases.empty:
        # Insert new column: scrubber ASN
        ases_copy.loc[:, 'scrubber'] = scrubber_asn

        # Put AS in front of AS numbers
        asn = ases['asn'].astype(str)
        
        ases_copy['ASN'] = 'AS' + ases['asn'].astype(str)

#         ases_copy.loc[:, 'ASN'] = "AS"+str(asn)
        dataframes.append(ases_copy)

# Combine all DataFrames into one
combined_df = pd.concat(dataframes)

asdb = pd.read_csv("/home/shyam/jupy/ddos_scrubber/data/2024-01_categorized_ases.csv", low_memory=False)
asdb_selected = asdb[["ASN", "Category 1 - Layer 1", "Category 1 - Layer 2"]]

# Merge our data with ASdb 
merged_df = combined_df.merge(asdb_selected, how='inner', on='ASN')
# Save the results to a new file (optional)
output_file = "../data/protected_ases_"+country_code+".csv"
merged_df.to_csv(output_file, index=False)

In [45]:
# Country specific AS catgories
# After TMA
import pandas as pd
import os
import re
import glob

# Group by countries
df = pd.read_csv("../data/after_tma/final_confirmed_customer_ases_location_01_dec_2024.csv")

# Group by org_country and count unique ASNs
asn_counts = df.groupby("org_country")["asn"].nunique().reset_index()

# Rename columns for clarity
asn_counts.columns = ["country", "asn_count"]

# Sort by count descending (optional)
asn_counts = asn_counts.sort_values(by="asn_count", ascending=False)

# Save to a new CSV
asn_counts.to_csv("final_confirmed_customer_ases_location_all_group__01_dec_2024.csv", index=False)

print("Saved final_confirmed_customer_ases_location_all_group__01_dec_2024.csv")
sorted_countries = asn_counts.sort_values(by = ['asn_count'], ascending=False)
print(sorted_countries[0:9])

# countries = pd.read_csv("../data/final_confirmed_customer_ases_location_all_group_2024.csv")
# sorted_countries = countries.sort_values(by = ['value'], ascending=False)
# sorted_countries[0:20]
# Top 8 countries are US, GB, AU, DE, HK, CA, SG, CH, FR.
# No. of ASNs on those countries are 676, 100, 82, 58, 51, 50, 33, 31.

# Define regex pattern
directory = "/home/shyam/jupy/ddos_scrubber/data/after_tma/"

# Initialize an empty list to store DataFrames
dataframes = []
country_code = "US"

df = pd.read_csv(directory + "final_confirmed_customer_ases_location_01_dec_2024.csv")

# Remove rows which do not have country code
filtered_df = df[~df['org_country'].str.contains(r'^(?:\?|.*[\t ]\?.*)', na=False)].copy() # Make a copy here

# Use .loc to avoid the warning and replace spaces in 'org_country' column
filtered_df.loc[:, 'org_country'] = filtered_df['org_country'].apply(lambda x: x.replace(' ', ''))

filtered_df[['org_country']] = filtered_df[['org_country']].apply(lambda x: x.astype(str).str[0:2])

# Remove tabs and spaces from column names
filtered_df.columns = filtered_df.columns.str.replace(r'[\t ]', '', regex=True)

ases = filtered_df[filtered_df["org_country"] == country_code]
ases_copy = ases.copy()

if not ases.empty:
    # Put AS in front of AS numbers
    asn = ases['asn'].astype(str)
    ases_copy['ASN'] = 'AS' + ases['asn'].astype(str)
    dataframes.append(ases_copy)

# Combine all DataFrames into one
combined_df = pd.concat(dataframes)

asdb = pd.read_csv("/home/shyam/jupy/ddos_scrubber/data/2024-01_categorized_ases.csv", low_memory=False)
asdb_selected = asdb[["ASN", "Category 1 - Layer 1", "Category 1 - Layer 2"]]

# Merge our data with ASdb 
merged_df = combined_df.merge(asdb_selected, how='inner', on='ASN')
# Save the results to a new file (optional)
output_file = "../data/after_tma/protected_ases_"+country_code+".csv"
merged_df.to_csv(output_file, index=False)

# See if the finance and insurance type of ASNs of the US covers the top banks of the US
fin_insurances = pd.read_csv("../data/after_tma/protected_ases_"+country_code+".csv")
fin_insurances = fin_insurances.loc[fin_insurances["Category 1 - Layer 1"] == "Finance and Insurance"]
fin_insurances[["org_name", "asn"]].to_csv("../data/after_tma/protected_ases_banks_"+country_code+".csv")


Saved final_confirmed_customer_ases_location_all_group__01_dec_2024.csv
          country  asn_count
95   US                  676
30   GB                  100
7    AU                   82
0    ?                    61
22   DE                   58
36   HK                   51
14   CA                   50
29   FR                   33
15   CH                   31


In [ ]:
# See if the ISP type of ASNs of the US covers the top banks of the US
country_code = "US"
import pandas as pd
fin_insurances = pd.read_csv("../data/protected_ases_"+country_code+".csv")
fin_insurances = fin_insurances.loc[fin_insurances["Category 1 - Layer 2"] == "Internet Service Provider (ISP)"]
fin_insurances[["org_name", "asn"]].to_csv("../data/protected_ases_isps_"+country_code+".csv")

In [ ]:
# Find all the protected ases of individual scrubbers and save it into a file
# customers_ases_scrubber_scrubber_asn_year.csv
import pandas as pd
import csv

# scrubber_asns = ["32787", "13335", "19551", "198949", "19905"]
year = "2024"
mon = "jan"

path = f"/home/shyam/jupy/ddos_scrubber/data/{year}/"

all_record_ases = []  # Stores number of customer ases for a month

for scrubber_asn in scrubber_asns:
    single_record_ases = []  # Stores number of customer ases for a month

    confirmed_customers1 = pd.read_csv(path + "confirmed_customers_as" + scrubber_asn + "_" + year + ".csv")
    confirmed_customers1_unique_origin_ases = confirmed_customers1['origin_as'].unique()

    if scrubber_asn != "13335":  # Cloudflare does not have path prepending case
        path_prepending = pd.read_csv(
            path + "unique_optimized_provider_as" + scrubber_asn + "_path_prepend_01_"+mon+"_" + year + ".csv")
        path_prepending_unique_origin_ases = path_prepending['origin_as'].unique()
        confirmed_customers2_ases = list(
            set(path_prepending_unique_origin_ases) - set(confirmed_customers1_unique_origin_ases))
    else:
        confirmed_customers2_ases = []

    if scrubber_asn != "13335":  # Cloudflare does not have path prepending case
        df = pd.read_csv(path + "unique_optimized_provider_not_as" + scrubber_asn + "_01_"+mon+"_" + year + "_v3.csv")
        sibling_path = df.loc[(df['siblings'] == 1) & (df['new_provider_sibling_check'] == int(scrubber_asn))]
        sibling_path_unique_origin_ases = sibling_path['origin_as'].unique()
    else:
        sibling_path_unique_origin_ases = []

    confirmed_customers3_ases = list(
        set(sibling_path_unique_origin_ases) - set(confirmed_customers2_ases) - set(
            confirmed_customers1_unique_origin_ases))

    confirmed_customers_ases = list(confirmed_customers1_unique_origin_ases) + list(
        confirmed_customers2_ases) + list(confirmed_customers3_ases)

    # Save to CSV file
    with open("/home/shyam/jupy/data/customers_ases_scrubber_" + scrubber_asn + "_"+mon+"_"+year+".csv", "w", newline="") as file:
        #     with open("../data/customers_ases_scrubber_"+scrubber_asn+"_2024_cloudflare.csv", "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["as"])  # Write header
        for ases in confirmed_customers_ases:
            writer.writerow([ases])  # Write each name in a new row
    print("Done for scrubber ", scrubber_asn)
print("Completed.")

In [ ]:
# Aruba: Find all the protected ases of individual scrubbers and save it into a file
# customers_ases_scrubber_scrubber_asn_mon_year.csv
import pandas as pd
import csv
import os

year = "2020"
months = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]

# List your scrubber ASNs here
scrubber_asns = ["32787", "13335", "19551", "198949", "19905"] 

for mon in months:
    print(f"Processing month: {mon}")
    
    for scrubber_asn in scrubber_asns:
        path = "/data/shared_dir/ddos_scrubber/as"+scrubber_asn+"/"+year+"/"
        all_record_ases = []  # Stores number of customer ases for a month

        # Load confirmed customers
        confirmed_customers1 = pd.read_csv(path + f"confirmed_customers_as{scrubber_asn}_01_{mon}_{year}.csv")
        confirmed_customers1_ases = confirmed_customers1['origin_as'].unique()
        
        if os.path.exists(path + "unique_optimized_provider_as" + scrubber_asn + "_path_prepend_01_" + mon + "_" + year + ".csv"):
            path_prepending = pd.read_csv(path + "unique_optimized_provider_as" + scrubber_asn + "_path_prepend_01_" + mon + "_" + year + ".csv")
            path_prepending_unique_origin_ases = path_prepending['origin_as'].unique()
            confirmed_customers2_ases = path_prepending_unique_origin_ases
        else:
            confirmed_customers2_ases = []
            
        if os.path.exists(path + "unique_optimized_provider_not_as" + scrubber_asn + "_01_" + mon + "_" + year + "_v3.csv"):
            df = pd.read_csv(
                path + "unique_optimized_provider_not_as" + scrubber_asn + "_01_" + mon + "_" + year + "_v3.csv")
            sibling_path = df.loc[(df['siblings'] == 1) & (df['new_provider_sibling_check'] == int(scrubber_asn))]
            sibling_path_unique_origin_ases = sibling_path['origin_as'].unique()
            confirmed_customers3_ases = sibling_path_unique_origin_ases

        else:
            confirmed_customers3_ases = []

        # Combine all unique origin ASes
        confirmed_customers_ases = set(confirmed_customers1_ases) | set(confirmed_customers2_ases) | set(confirmed_customers3_ases)  # Union

        print(f"Customers 1: {len(confirmed_customers1_ases)}, 2: {len(confirmed_customers2_ases)}, 3: {len(confirmed_customers3_ases)}, final {len(confirmed_customers_ases)}")
        # Write to CSV
        output_path = f"{path}customers_ases_scrubber_{scrubber_asn}_{mon}_{year}.csv"
        os.makedirs(os.path.dirname(output_path), exist_ok=True)

        with open(output_path, "w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(["as"])  # Header
            for ases in confirmed_customers_ases:
                writer.writerow([ases])

        print(f"Done for scrubber {scrubber_asn} in {mon}")
print("Completed all months.")

In [ ]:
# Find total protected ases of all scrubbers
import os
import re
import pandas as pd

year = "2017"
 
pattern = r"^customers_ases_scrubber_\d+_"+year+"\.csv$" #customers_ases_scrubber_19905_2023.csv
directory = "/home/shyam/jupy/ddos_scrubber/data"

csv_files = [f for f in os.listdir(directory) if re.match(pattern, f)]

# Initialize an empty list to store DataFrames
dataframes = []

# Read each CSV file and append it to the list
for file in csv_files:
    df = pd.read_csv(directory+"/"+file)
    dataframes.append(df)

# Merge all dataframes into one
merged_df = pd.concat(dataframes, ignore_index=True)

# Save the merged dataframe into a new CSV file
merged_df.to_csv("../data/customers_ases_scrubber_all_"+year+".csv", index=False) 

merged_df_unique = merged_df.drop_duplicates()

# Save the merged dataframe into a new CSV file
merged_df_unique.to_csv("../data/customers_ases_unique_scrubber_all_"+year+".csv", index=False) 
print(len(merged_df_unique))

print("Completed. All the ASes of %s saved in file customers_ases_scrubber_all_%s.csv"  %(year, year))

In [ ]:
# Find total protected ases of all scrubbers monthwise
# After TMA
import os
import re
import pandas as pd

years = ["2020", "2021", "2022", "2023", "2024"]
day = "01"
months = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]

# List your scrubber ASNs here
scrubber_asns = ["32787", "13335", "19551", "198949", "19905"]

for year in years:
    
    for mon in months:
        print(f"🔄 Processing month: {mon}")

        all_matching_files = []

        for scrubber in scrubber_asns:
            base_dir = f"../data/after_tma/as{scrubber}"

            # Regex to match files like customers_ases_scrubber_13335_01_jan_2024.csv
            pattern = rf"^customers_ases_scrubber_\d+_{day}_{mon}_{year}\.csv$"

            for root, dirs, files in os.walk(base_dir):
                for file in files:
                    if re.match(pattern, file):
                        full_path = os.path.join(root, file)
                        all_matching_files.append(full_path)

        # Merge all matching files
        if all_matching_files:
            dataframes = [pd.read_csv(f) for f in all_matching_files]
            merged_df = pd.concat(dataframes, ignore_index=True)

            print(f"✅ Without removing duplicates: {len(merged_df)}")
            # Optional: Drop duplicates
            merged_df_unique = merged_df.drop_duplicates()

            # Output path
            output_file = f"../data/after_tma/customers_ases_scrubber_all_{day}_{mon}_{year}.csv"
            merged_df_unique.to_csv(output_file, index=False)
            print(f"📁 Saved merged file: {output_file} ({len(merged_df_unique)} unique rows)")
        else:
            print(f"⚠️ No files found for {mon} {year}")

    print(f"✅ Done for all months for year .{year}")


In [2]:
# Method to get AS rank of all ASes from CAIDA AS rank API
# This part of the code is used from https://github.com/bgpkit/pyasrank

import json
import logging
from datetime import datetime

import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

ASRANK_ENDPOINT = "https://api.asrank.caida.org/v2/graphql"


def ts_to_date_str(ts):
    """
    Convert timestamp to a date. This is used for ASRank API which only takes
    date strings with no time as parameters.api
    """
    return datetime.utcfromtimestamp(int(ts)).strftime("%Y-%m-%d")


class AsRank:
    """
    Utilities for using ASRank services
    """

    def __init__(self, max_ts=""):
        self.data_ts = None

        # various caches to avoid duplicate queries
        self.cache = None
        self.cone_cache = None
        self.neighbors_cache = None
        self.siblings_cache = None
        self.organization_cache = None

        self.queries_sent = 0

        self.session = None
        self._initialize_session()

        self.init_cache(max_ts)

    def _initialize_session(self):
        self.session = requests.Session()
        retries = Retry(total=5,
                        backoff_factor=1,
                        status_forcelist=[500, 502, 503, 504])
        self.session.mount(ASRANK_ENDPOINT, HTTPAdapter(max_retries=retries))

    def _close_session(self):
        if self.session:
            self.session.close()

    def _send_request(self, query):
        """
        send requests to ASRank endpoint
        :param query:
        :return:
        """

        r = self.session.post(url=ASRANK_ENDPOINT, json={'query': query})
        r.raise_for_status()
        self.queries_sent += 1
        return r

    def init_cache(self, ts):
        """
        Initialize the ASRank cache for the timestamp ts
        :param ts:
        :return:
        """
        self.cache = {}
        self.cone_cache = {}
        self.neighbors_cache = {}
        self.siblings_cache = {}
        self.organization_cache = {}
        self.queries_sent = 0
        if isinstance(ts, int):
            ts = ts_to_date_str(ts)

        ####
        # Try to cache datasets available before the given ts
        ####
        graphql_query = """
            {
              datasets(dateStart:"2000-01-01", dateEnd:"%s", sort:"-date", first:1){
                edges {
                  node {
                    date
                  }
                }
              }
            }
        """ % ts
        r = self._send_request(graphql_query)

        edges = r.json()['data']['datasets']['edges']
        if edges:
            self.data_ts = edges[0]["node"]["date"]
            return

        # if code reaches here, we have not found any datasets before ts. we should now try to find one after ts.
        # this is the best effort results
        logging.warning("cannot find dataset before date %s, looking for the closest one after it now" % ts)

        graphql_query = """
            {
              datasets(dateStart:"%s", sort:"date", first:1){
                edges {
                  node {
                    date
                  }
                }
              }
            }
        """ % ts
        r = self._send_request(graphql_query)
        edges = r.json()['data']['datasets']['edges']
        if edges:
            self.data_ts = edges[0]["node"]["date"]
            logging.warning("found closest dataset date to be %s" % self.data_ts)
            return
        else:
            raise ValueError("no datasets from ASRank available to use for tagging")

    def _query_asrank_for_asns(self, asns, chunk_size=100):
        asns = [str(asn) for asn in asns]
        asns_needed = [asn for asn in asns if asn not in self.cache]
        if not asns_needed:
            return

        # https://stackoverflow.com/a/312464/768793
        def chunks(lst, n):
            """Yield successive n-sized chunks from lst."""
            for i in range(0, len(lst), n):
                yield lst[i:i + n]

        for asns in chunks(asns_needed, chunk_size):

            graphql_query = """
                {
                  asns(asns: %s, dateStart: "%s", dateEnd: "%s", first:%d, sort:"-date") {
                    edges {
                      node {
                        date
                        asn
                        asnName
                        rank
                        organization{
                          country{
                            iso
                            name
                          }
                          orgName
                          orgId
                        } asnDegree {
                          provider
                          peer
                          customer
                          total
                          transit
                          sibling
                        }
                      }
                    }
                  }
                }
            """ % (json.dumps(asns), self.data_ts, self.data_ts, len(asns))
            r = self._send_request(graphql_query)
            try:
                for node in r.json()['data']['asns']['edges']:
                    data = node['node']
                    if data['asn'] not in self.cache:
                        if "asnDegree" in data:
                            degree = data["asnDegree"]
                            degree["provider"] = degree["provider"] or 0
                            degree["customer"] = degree["customer"] or 0
                            degree["peer"] = degree["peer"] or 0
                            degree["sibling"] = degree["sibling"] or 0
                            data["asnDegree"] = degree
                        self.cache[data['asn']] = data
                for asn in asns:
                    if asn not in self.cache:
                        self.cache[asn] = None
            except KeyError as e:
                logging.error("Error in node: {}".format(r.json()))
                logging.error("Request: {}".format(graphql_query))
                raise e

   
    ###########
    # AS_RANK #
    ###########

    def get_degree(self, asn):
        """
        Get relationship summary for asn, including number of customers, providers, peers, etc.

        Example return dictionary:
        {
            "provider": 0,
            "peer": 31,
            "customer": 1355,
            "total": 1386,
            "transit": 1318,
            "sibling": 25
        }
        :param asn:
        :return:
        """
        self._query_asrank_for_asns([asn])
        if self.cache[asn] is None:
            return None

        return self.cache[asn]["asnDegree"]

      
    def cache_asrank_chunk(self, asns: list, chunk_size: int):
        """
        Query asrank info in chunk to boost individual asrank queries performance later.

        :param asns:
        :param chunk_size:
        :return:
        """
        self._query_asrank_for_asns(asns, chunk_size)  

In [ ]:
# Find relationship between ASes on an AS Path
as_rank = AsRank("2024-01-01")
a = as_rank.get_degree("23752")["customer"]
a

In [ ]:
customer = a["customer"]
a

In [ ]:
# Check how many of them are stub
import pandas as pd
df = pd.read_csv("../data/customers_ases_unique_scrubber_all_2024.csv")
ases = df["as"]

transit_counter = 0
for asn in ases:
    # Check customer AS using ASRank
    as_rank = AsRank("2024-01-01")
    asn_degree = as_rank.get_degree(str(asn))
    customer_count = asn_degree["customer"]
    if customer_count > 0:
        stub_counter += 1
        print(f"Degree of AS{asn} is {asn_degree}.")
print(transit_counter)    

In [3]:
# Check how many of them are stub
# After TMA

import pandas as pd
df = pd.read_csv("../data/after_tma/customers_ases_scrubber_all_01_dec_2024.csv")
ases = df["as"]

transit_counter = 0
stub_counter = 0

for asn in ases:
    # Check customer AS using ASRank
    as_rank = AsRank("2024-12-01")
    asn_degree = as_rank.get_degree(str(asn))
    customer_count = asn_degree["customer"]
    if customer_count > 0:
        transit_counter += 1
        print(f"Degree of AS{asn} is {asn_degree}.")
print(transit_counter)    

Degree of AS3 is {'provider': 11, 'peer': 3, 'customer': 2, 'total': 16, 'transit': 3, 'sibling': 0}.
Degree of AS28690 is {'provider': 4, 'peer': 0, 'customer': 2, 'total': 6, 'transit': 6, 'sibling': 1}.
Degree of AS12357 is {'provider': 2, 'peer': 1, 'customer': 95, 'total': 98, 'transit': 96, 'sibling': 1}.
Degree of AS4199 is {'provider': 3, 'peer': 0, 'customer': 2, 'total': 5, 'transit': 4, 'sibling': 1}.
Degree of AS6303 is {'provider': 3, 'peer': 0, 'customer': 2, 'total': 5, 'transit': 5, 'sibling': 0}.
Degree of AS24756 is {'provider': 1, 'peer': 0, 'customer': 1, 'total': 2, 'transit': 2, 'sibling': 0}.
Degree of AS47388 is {'provider': 8, 'peer': 0, 'customer': 1, 'total': 9, 'transit': 3, 'sibling': 0}.
Degree of AS43338 is {'provider': 5, 'peer': 2, 'customer': 2, 'total': 9, 'transit': 9, 'sibling': 1}.
Degree of AS2386 is {'provider': 5, 'peer': 1, 'customer': 9, 'total': 15, 'transit': 10, 'sibling': 0}.
Degree of AS14733 is {'provider': 4, 'peer': 0, 'customer': 1, '

Degree of AS136462 is {'provider': 6, 'peer': 0, 'customer': 1, 'total': 7, 'transit': 2, 'sibling': 0}.
Degree of AS5408 is {'provider': 3, 'peer': 7, 'customer': 30, 'total': 40, 'transit': 38, 'sibling': 0}.
Degree of AS54574 is {'provider': 1, 'peer': 4, 'customer': 1, 'total': 6, 'transit': 4, 'sibling': 1}.
Degree of AS5430 is {'provider': 5, 'peer': 110, 'customer': 2, 'total': 117, 'transit': 114, 'sibling': 0}.
Degree of AS7483 is {'provider': 14, 'peer': 8, 'customer': 1, 'total': 23, 'transit': 12, 'sibling': 0}.
Degree of AS13649 is {'provider': 5, 'peer': 1, 'customer': 173, 'total': 179, 'transit': 177, 'sibling': 0}.
Degree of AS7506 is {'provider': 4, 'peer': 18, 'customer': 4, 'total': 26, 'transit': 23, 'sibling': 3}.
Degree of AS216425 is {'provider': 6, 'peer': 0, 'customer': 1, 'total': 7, 'transit': 2, 'sibling': 0}.
Degree of AS23930 is {'provider': 8, 'peer': 43, 'customer': 6, 'total': 57, 'transit': 49, 'sibling': 1}.
Degree of AS1421 is {'provider': 5, 'peer'

In [4]:
len(ases)

1730

In [ ]:
df = pd.read_csv("../data/customers_ases_unique_scrubber_all_2024.csv")
ases = df["as"]
len(ases)

In [ ]:
# Out of total ISPs, check how many of them are stub and transit ASes
# Category 1 - Layer 2 is an ISP
import pandas as pd
df = pd.read_csv("../data/customers_ases_scrubber_all_2024.csv")
as_list = df["as"].unique()


# Load ASdb CSV file into a pandas DataFrame
asdb = pd.read_csv("/home/shyam/jupy/ddos_scrubber/data/"+year+"-01_categorized_ases.csv", low_memory=False)

# Ensure AS numbers in the CSV are formatted correctly as strings and strip extra spaces
asdb['ASN'] = asdb['ASN'].astype(str).str.strip()

# # Put "AS" prefix prepend as the file contains only numbers
as_list = [f'AS{asn}' for asn in as_list]

# Filter ASNs with category 'ISP' that are in the list
isp_ases = asdb[(asdb['Category 1 - Layer 2'] == 'Internet Service Provider (ISP)') & (asdb['ASN'].isin(as_list))]['ASN'].tolist()

# Remove the "AS" prefix
ases = [as_num.replace("AS", "") for as_num in isp_ases]

# Check how many of them are stub and transit
as_rank = AsRank("2024-01-01")
# Check how many of them are stub
transit_counter = 0
for asn in ases:
    # Check customer AS using ASRank
    as_rank = AsRank("2024-01-01")
    asn_degree = as_rank.get_degree(str(asn))
    customer_count = asn_degree["customer"]
    if customer_count > 0:
        transit_counter += 1
#         print(f"Degree of AS{asn} is {asn_degree}.")
print(transit_counter) 

In [ ]:
list1 = [1, 2, 3, 4]
list2 = [3, 4, 5]
list3 = [4, 6, 7]

all_unique_values = set(list1) | set(list2) | set(list3)  # Union

print("Values appearing in at least one list:", all_unique_values)
